This notebook analyzes model predictions for only the battleground states.

In [1]:
import os
import sys

import numpy as np
import pandas as pd

In [2]:
BATTLEGROUNDS = {
    'AZ': 11,
    'FL': 29,
    'GA': 16,
    'IA': 6,
    'ME2': 1, 
    'MI': 16,
    'ME': 2,
    'NE2': 1,
    'NV': 6,
    'NH': 4,
    'NC': 15,
    'OH': 18,
    'PA': 20,
    'TX': 38,
    'WI': 10
}

def get_evs(abbr):
    return BATTLEGROUNDS.get(abbr)

In [3]:
DATA_DIR = "./data"

In [4]:
WINNERS_PATH = os.path.join(DATA_DIR, 'winners.csv')
winners = pd.read_csv(WINNERS_PATH)
winners['ev'] = winners.apply(lambda r: get_evs(r.state), axis=1)

In [5]:
winners.head()

,office,state,winner,dem_diff,dem_share,dem_share_2p,candidates,ev
0,P,AZ,Biden,0.003,0.494,0.501523,BIDEN|TRUMP,11.0
1,P,FL,Trump,-0.034,0.479,0.483350,BIDEN|TRUMP,29.0
2,P,GA,Biden,0.003,0.495,0.501012,BIDEN|TRUMP,16.0
3,P,IA,Trump,-0.082,0.449,0.458163,BIDEN|TRUMP,6.0
4,P,ME2,Trump,-0.079,0.447,0.458932,BIDEN|TRUMP,1.0


In [6]:
source_dfs = []
for fn in os.listdir(os.path.join(DATA_DIR, 'state-level')):
    print(fn)
    df = pd.read_csv(os.path.join(DATA_DIR, 'state-level', fn))
    df = df[df['date'] == df['date'].max()]
    source_dfs.append(df)
state_forecasts = pd.concat(source_dfs, axis=0)
state_forecasts.shape

pollyvote.csv
fivethirtyeight.csv
northwestern.battleground_only.csv
economist.csv
pec.csv
uva.csv


(608, 13)

In [7]:
state_forecasts = state_forecasts[
    (state_forecasts["office"] == 'P') &
    (state_forecasts['state'].isin(BATTLEGROUNDS.keys()))]
state_forecasts

,date,model,office,state,party,candidate,win_prob,est_diff,est_share,est_share_2p,Unnamed: 0,index,prediction
2,2020-11-03,pollyvote,P,NH,D,Biden,0.87,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-03,pollyvote,P,NH,R,Trump,0.13,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-03,pollyvote,P,ME,D,Biden,0.95,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-11-03,pollyvote,P,ME,R,Trump,0.05,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-11-03,pollyvote,P,NV,D,Biden,0.82,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2020-10-27,UVA,P,PA,D,Biden,0.80,NaN,NaN,NaN,77.0,43.0,favored
86,2020-10-27,UVA,P,TX,D,Biden,0.40,NaN,NaN,NaN,86.0,104.0,not lean
87,2020-10-27,UVA,P,TX,R,Trump,0.60,NaN,NaN,NaN,87.0,48.0,lean
96,2020-10-27,UVA,P,WI,R,Trump,0.20,NaN,NaN,NaN,96.0,109.0,not favored


In [8]:
def get_credits(race):
    max_prob = race["win_prob"].max()
    at_max = race["win_prob"] == max_prob
    favorites = race[at_max]
    credit = (1 / len(favorites)) * favorites["correct"]
    return credit.sum()

def brier_score_race_statelevel(called_forecast):
    uniques = called_forecast[[
        "date", "office", "state", "model"
    ]].apply(lambda x: x.nunique())
    assert((uniques != 1).sum() == 0)
    errors = called_forecast["win_prob"] - called_forecast["correct"]
    errors_squared = (errors).pow(2).sum()
    # If you didn't put odds on the candidate, add 1
    if called_forecast["correct"].sum() == 0:
        errors_squared += 1
    return errors_squared / 2  # divide by 2 bc there are 2 forecasts: Biden and Trump

In [9]:
def forecast_scores_statelevel(forecasts):
    called = pd.merge(winners, forecasts, on=['office', 'state'], how='left').dropna(
        subset=['winner', 'win_prob'])
    called['correct'] = called['winner'] == called['candidate']
    
    scores = pd.merge(
        called, forecasts[['date', 'model']].drop_duplicates(),
        how='inner', on=['date', 'model']
    ).groupby(['date', 'office', 'state', 'model', 'ev']).apply(brier_score_race_statelevel)\
            .reset_index()\
            .rename(columns={0: 'brier_score'})
    scores.loc[((scores['office'] == 'P')), 'brier_evs'] = scores.apply(
        lambda x: x['brier_score'] * get_evs(x['state']), axis=1)
    credits = pd.DataFrame({
        "credit": called.groupby([
                "date", "model", "office", "state"
            ]).apply(get_credits)
    }).reset_index()
    
    evs_called = scores.groupby('model')['ev'].sum()
    scores = pd.DataFrame({
        "brier_score": scores.groupby("model")["brier_score"].mean(),
        "ev-weighted_brier": scores.groupby("model")["brier_evs"].sum() / evs_called,
        "accuracy": credits[credits["office"] == "P"].groupby("model")["credit"].sum()\
                        / scores.groupby("model").size(),
    })
    return scores
    

In [10]:
forecast_scores_statelevel(state_forecasts)

,brier_score,ev-weighted_brier,accuracy
model,,,
UVA,0.132308,0.169215,0.846154
economist,0.143917,0.197716,0.846154
fivethirtyeight-polls-plus,0.133959,0.182416,0.846154
northwestern,0.153107,0.208346,0.833333
pec,0.148807,0.174368,0.800000
pollyvote,0.121027,0.156319,0.846154
